# Modelado de tópicos con BERTopic


Para compreder acerca de qué temas se habla en un conjunto de documentos, es necesario realizar un análisis de tópicos. El análisis de tópicos es una técnica de minería de texto que permite identificar los temas principales que se tratan en un conjunto de documentos. Exiten una gran cantidad de algoritmos que permiten realizar este análisis, entre ellos destacan *Latent Dirichlet Allocation, Latent Semantic Analysis, Non-negative Matrix Factorization*, entre otros.

Siendo la mayoria de estos métodos basados en la representación de los documentos como vectores de palabras, es decir, se representan los documentos como vectores de frecuencias de palabras, lo que implica que se pierde la información de contexto de las palabras.

En el año 2022 se lanzó [BERTopic](https://arxiv.org/abs/2203.05794), un algoritmo de análisis de tópicos basado en la representación de los documentos como vectores de palabras, pero utilizando la representación de palabras de cualquier modelo transformer de lenguaje (como BERT), a modo que la información contextual contenida dentro de cada texto sea aprovechada. BERTopic es un algoritmo de análisis de tópicos auto-,semi- y supervisado, lo que significa que no es necesario etiquetar todos los documentos para realizar el análisis de tópicos.

El modelo de BERTopic permite agrupar textos cuya semántica es similar y provee de una jerarquía de palabras simples o compuestas para cada grupo de textos. Esta jerarquía de palabras permite al usuario asociar un tema o conjunto de temas a cada grupo.
El funcionamiento de este modelo o algoritmo se divide en tres etapas:

* **Generación y extracción de embbedings por documento**: En esta etapa se utiliza un modelo transformer de lenguaje para generar los embeddings de cada documento. Los embeddings son vectores de números reales que representan la información contextual de cada documento.

* **Agrupamiento de los vectores de documentos en grupos semánticamente similares**: En esta etapa se utiliza un algoritmo de clustering para agrupar los vectores de documentos en grupos semánticamente similares. El algoritmo de clustering utilizado es DBSCAN, el cual permite identificar grupos de documentos que se encuentran cercanos entre sí y que no se encuentran cercanos a los grupos de otros documentos.

* **Creación de la representación de palabras o tópicos para cada grupo**: Para asignar a cada grupo un conjunto de palabras claves, los textos asignados a cada grupo son considerados como un único documento, para el cual se mide la importancia de las palabras contenidas a partir de la frecuencia de aparición. Esto se realiza gracias a la medida estadística conocida como *Term frequency – Inverse document frequency* (Tf-Idf). De esta manera, a partir de las palabras más importantes de cada grupo se obtiene una descripción del tópico.

En este notebook se presenta un ejemplo de aplicación de BERTopic semisupervisado para el análisis de tópicos de un conjunto de documentos. Se presentarán algunas gráficas que permiten visualizar algunos hitos dentro de los datos usados.

In [1]:
#Imports

#models
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

#data loading and manipualtion
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/pedri0/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pedri0/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Carga de datos etiquetados y no etiquetados sobre migración en twitter

Se leerán los datos para modelado de topicos a través del modelo BERTopic, el cual será entrenado de manera semisupervisada. El conjunto de datos contiene etiquetas para los temas covid (477), servicios públicos (465), género (464), seguridad/crimen (429), salud (188) y empleo (427). El conjunto de datos contiene 10,000 tweets. Los textos no etiquetados contienen la etiqueta -1.

In [2]:
#read data
data = pd.read_csv('assets/data/bertopic_train.csv')
#convert string labels to int labels
target_label = {'covid': 0, 'servpub':1, 'genero':2, 'crimen':3, 'empleo':4, 'salud':5, '-1':int(-1)}
data['target'] = data.label.apply(lambda x: target_label[x])
data = data.sample(frac=0.4).reset_index(drop=True)

### Configuración del modelo BERTopic

Para la etapa de generación de embbedings, se usa un modelo preentrenado en español. Dicho modelo puede intercambiarse por alguno otro más eficiente o entrenado para otro idioma. Los modelos disponibles pueden encontrarse en la página de la paqueteria [SBERT](https://www.sbert.net/docs/pretrained_models.html) o bien en la página de [Hugging Face](https://huggingface.co/models). Para fines demostrativos en este notebook se emplea un modelo eficiente en cuanto a tiempo de inferencia y memoria, entrenado en español llamado *paraphrase-multilingual-MiniLM-L12-v2*.

Tanto para el agrupamiento y la representación de palabras, se usa un modelo que permite eliminar las palabras vacías,  determinar la cantidad de palabras dentro de un grupo a analizar (n-gramas) e ignorar palabras poco comunes.


In [3]:
#add our stop word through countvectorizer model, set to 2 the limit of n-grams
# with min_df: ignore terms that have a document frequency strictly lower than the given threshold (5)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('spanish'), ngram_range=(2,2), min_df=1)

#use paraphrase-multilingual-mpnet-base-v2 as embedding model. This model is the most accurate for multilingual
#diversity is a parameter in BERTopic to diversity words in each topic such that we limit the number of 
#duplicate words we find in each topic. This is done using an algorithm called Maximal Marginal Relevance 
#which compares word embeddings with the topic embedding.
#We do this by specifying a value between 0 and 1, with 0 being not at all diverse and 1 being completely diverse
#min_topic_size is used to specify what the minimum size of a topic can be. 
#The lower this value the more topics are created.
#calculate_probabilities lets you calculate the probabilities of each topic to each document.
topic_model = BERTopic(embedding_model='paraphrase-multilingual-MiniLM-L12-v2', verbose=True,
                      vectorizer_model=vectorizer_model, diversity=0.5, min_topic_size=5,
                      calculate_probabilities=False)

### Entrenamiento del modelo BERTopic

Para el entrenamiento semisupervisado, los datos de entrada no etiquetadas deben contener la etiqueta -1. Mientras que las etiquetas deben ser números enteros. Para los casos supervisado y semisupervisado estas etiquetas son un parámetro y del método fit. Para el caso auto-supervisado, este parámetro no se usa.

Según la cantidad de datos, el tiempo de entrenamiento puede variar significativamente pues dos de los tres modelos involucrados no están optimizados para GPU.

Con el método fit_transform se obtiene una lista de números enteros que representan el grupo al que pertenece cada documento.

In [4]:
#fit the model using semisupervised topic modeling and infer clusters, probabilities
topics = topic_model.fit_transform(data.text.to_list(), y=data.target.tolist())

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2022-10-17 15:24:33,684 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-10-17 15:24:43,859 - BERTopic - Reduced dimensionality
2022-10-17 15:24:43,940 - BERTopic - Clustered reduced embeddings


In [5]:
#save inferred clusters of topics
data['cluster_topic'] = topics[0]
#save the trained model
#topic_model.save("bertopic_20news.model")

Una vez entranado el modelo pueden obtenerse las palabras más representativas de cada grupo, así como la cantidad de documentos que pertenecen a cada grupo. Esto se hace con el método ``get_topic_info()``.

In [6]:
#get dataframe with info about clusters from fitted model
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,870,-1_derechos humanos_migrantes venezolanos_cier...
1,0,58,0_gracioso tiro_racista si_prender fuego_2020 ...
2,1,54,1_inmigrantes venezolanos_800 venezuela_especi...
3,2,49,2_venezolanos delincuentes_inseguridad ciudada...
4,3,33,3_cerrar fronteras_ingreso extranjeros_fronter...
...,...,...,...
64,63,5,63_ingreso extranjeros_admiten extranjeros_hot...
65,64,5,64_pc ps_presidente2022 delincuentes_vamos nec...
66,65,5,65_16 anos_radicales abordar_pf rt_solicitud e...
67,66,5,66_republica dominicana_agente enemigo_migraci...


Para obtener las 10 primeras palabras más representativas de cada grupo, así como sus probabilidades de pertenencia, se usa el método ``get_topics()``.

In [7]:
#get top 10 words per topic
topic_model.get_topics()

{-1: [('derechos humanos', 0.002875739873487939),
  ('migrantes venezolanos', 0.0025453705730600195),
  ('cierre fronteras', 0.0020888545781166507),
  ('caravana migrante', 0.001707659531177889),
  ('crisis migratoria', 0.0016002742315714398),
  ('refugiados venezolanos', 0.0015991711235530258),
  ('mas inmigrantes', 0.0015423365804628415),
  ('30 anos', 0.0015423365804628415),
  ('venezolanos colombia', 0.0014114170251951633),
  ('dejar entrar', 0.0013332392023639788)],
 0: [('gracioso tiro', 0.029571600742905654),
  ('racista si', 0.02131726507240328),
  ('prender fuego', 0.012031986419332836),
  ('2020 pasar', 0.012031986419332836),
  ('protestas sucesos', 0.012031986419332836),
  ('querran unirse', 0.012031986419332836),
  ('quede casa', 0.012031986419332836),
  ('quemar racista', 0.012031986419332836),
  ('racismo inevitable', 0.012031986419332836),
  ('quiere genocidio', 0.012031986419332836)],
 1: [('inmigrantes venezolanos', 0.02734096473254114),
  ('800 venezuela', 0.017156134

Una forma más visual de observar una cantidad determinada de grupos y la cantidad de palabras más representativas puede hacerse con el método ``visualize_topics()``.

In [8]:
topic_model.visualize_barchart(topics=[1,3,5,7,9,11,13,15,17,19], n_words=8, width=450, height=400)

A través de los embbedings y los tópicos es posible estimar la jerarquía entre los grupos. Esto se puede hacer con el método ``get_topic_hierarchy()``. La jerarquía es útil para comprender la relación entre los grupos y para identificar grupos que se encuentran dentro de otros grupos.

In [9]:
topic_model.visualize_hierarchy()

### Búsqueda de tópicos por palabra clave
Pueden obtenerse los tópicos que contienen una palabra clave o son similares a dicha plabra con el método ``find_topics()``. Este método el número de tópicos así como su índice de similitud.

In [18]:
query_word = 'covid'
similar_topics, similarity = topic_model.find_topics(query_word, top_n=5)
print('Los temas similares a {} son: {}, cuya similitud es {}'.format(query_word, similar_topics, similarity))

Los temas similares a covid son: [46, 36, 62, 7, 33], cuya similitud es [0.5719180231678345, 0.5647417562028396, 0.5413615662729051, 0.5411579846567005, 0.5410703588482876]


Mostremos una muestra aleatoria de cinco documentos del grupo más similar a la palabra clave.

In [19]:
data[data.cluster_topic==similar_topics[0]].sample(5)

,_id,label,a_code,text,target,cluster_topic
859,1301562760044720129,-1,NaN,pues en otros paises ni la han llevado la masc...,-1,46
1754,1385985184701587466,-1,CRI,recuerdan cuando zepolito pedia a otros revisa...,-1,46
975,1481630797534842880,covid,CHL,"exacto..no entiendo porque cerrar fronteras, s...",0,46
1894,1341151351619919878,covid,NaN,"nada mas sensato, el virus antes de contagiart...",0,46
1712,1410195942645633025,covid,NaN,che alvaro si quienes trajeron el virus son lo...,0,46
